In [1]:
class Persona:
    def __init__(self, nombre, contacto):
        self.nombre = nombre
        self.contacto = contacto

class Cliente(Persona):
    def __init__(self, nombre, contacto, nivel_de_fidelidad):
        super().__init__(nombre, contacto)
        self.nivel_de_fidelidad = nivel_de_fidelidad
        self.historial_pedidos = []

    def realizar_pedido(self, pedido):
        self.historial_pedidos.append(pedido)
        print(f"Pedido realizado por {self.nombre}")

    def consultar_historial(self):
        if self.historial_pedidos:
            print(f"Último pedido de {self.nombre}: {self.historial_pedidos[-1]}")
        else:
            print(f"{self.nombre} no tiene pedidos.")

    def obtener_descuento(self):
        descuentos = {"Cliente Casual": 0.05, "Cliente Leal": 0.10, "Café Adicto": 0.15}
        descuento = descuentos.get(self.nivel_de_fidelidad, 0.0)
        print(f"Descuento aplicado ({self.nivel_de_fidelidad}): {descuento*100}%")
        return descuento

class Empleado(Persona):
    def __init__(self, nombre, contacto, rol):
        super().__init__(nombre, contacto)
        self.rol = rol

    def actualizar_inventario(self, inventario, producto, cantidad):
        inventario.actualizar_stock(producto, cantidad)
        print(f"{self.nombre} actualizó el stock de {producto} (+{cantidad})")

    def cambiar_estado_pedido(self, pedido, estado):
        pedido.actualizar_estado(estado)
        print(f"{self.nombre} cambió el estado del Pedido a {estado}")

class ProductoBase:
    def __init__(self, nombre_producto, precio):
        self.nombre_producto = nombre_producto
        self.precio = precio

    def calcular_precio(self):
        return self.precio

class Bebida(ProductoBase):
    def __init__(self, nombre_producto, precio, tamaño, tipo):
        super().__init__(nombre_producto, precio)
        self.tamaño = tamaño
        self.tipo = tipo
        self.personalizacion = {}

    def personalizar(self, opcion, valor):
        self.personalizacion[opcion] = valor
        print(f"Bebida {self.nombre_producto} personalizada: {opcion} = {valor}")

class Postre(ProductoBase):
    def __init__(self, nombre_producto, precio, vegano, sin_gluten):
        super().__init__(nombre_producto, precio)
        self.vegano = vegano
        self.sin_gluten = sin_gluten

class Inventario:
    def __init__(self, stock_inicial):
        self.stock = stock_inicial
        print(f"Inventario inicializado: {self.stock}")

    def actualizar_stock(self, producto, cantidad):
        self.stock[producto] = max(0, self.stock.get(producto, 0) + cantidad)
        print(f"Stock {producto}: {self.stock[producto]}")

    def consultar_disp(self, producto):
        disponible = self.stock.get(producto, 0) > 0
        print(f"{producto}: {'Disponible' if disponible else 'Sin stock'}")
        return disponible

class Pedido:
    def __init__(self, cliente):
        self.cliente = cliente
        self.productos = []
        self.estado = "pendiente"
        print(f"Pedido creado para {self.cliente.nombre}")

    def agregar_producto(self, producto, inventario):
        if inventario.consultar_disp(producto.nombre_producto):
            self.productos.append(producto)
            inventario.actualizar_stock(producto.nombre_producto, -1)
            print(f"{producto.nombre_producto} agregado al Pedido")
        else:
            print(f"Sin stock: {producto.nombre_producto}")

    def calcular_total(self):
        total = sum(producto.calcular_precio() for producto in self.productos)
        print(f"Total del Pedido: ${total:.2f}")
        return total

    def actualizar_estado(self, nuevo_estado):
        self.estado = nuevo_estado
        print(f"Estado del Pedido cambiado a: {nuevo_estado}")

    def modificar_personalizacion(self, producto, opcion, valor):
        if isinstance(producto, Bebida):
            producto.personalizar(opcion, valor)
            print(f"Personalización modificada: {producto.nombre_producto} ({opcion} = {valor})")

    def obtener_resumen(self):
        total = self.calcular_total()
        resumen = {
            "Cliente": self.cliente.nombre,
            "Productos": [p.nombre_producto for p in self.productos],
            "Total": f"${total:.2f}",
            "Estado": self.estado
        }
        print(f"Resumen del Pedido: {resumen}")
        return resumen

class Promocion:
    def __init__(self, descuento, aplicable_a):
        self.descuento = descuento
        self.aplicable_a = aplicable_a
        print(f"Promoción creada: {descuento}% en {aplicable_a}")

    def aplicar_descuento(self, pedido):
        total = pedido.calcular_total()
        total_descuento = total * (1 - self.descuento / 100)
        print(f"Descuento aplicado: Nuevo total ${total_descuento:.2f}")
        return total_descuento

Ejemplo de uso del programa

In [2]:
# Crear inventario
inventario = Inventario({
        "Café Americano": 5,
        "Latte": 3,
        "Brownie": 4
    })

# Crear clientes
cliente1 = Cliente("Lidia Montiel", "555-666-1212", "Cliente Leal")
cliente2 = Cliente("Daniel Sánchez", "555-777-2323", "Café Adicto")

# Crear empleado
empleado = Empleado("Adrián Morales", "555-454-3345", "Mesero")

# Crear productos
cafe_americano = Bebida("Café Americano", 30, 250, "Caliente")
latte = Bebida("Latte", 45, 350, "Caliente")
brownie = Postre("Brownie", 25, False, False)

# Personalizar bebida
latte.personalizar("leche", "almendra")
latte.personalizar("azúcar", "sin azúcar")

# Crear pedidos con productos
pedido1 = Pedido(cliente1)
pedido1.agregar_producto(cafe_americano, inventario)
pedido1.agregar_producto(brownie, inventario)

pedido2 = Pedido(cliente2)
pedido2.agregar_producto(latte, inventario)
pedido2.agregar_producto(brownie, inventario)

# Aplicar descuento
descuento_cliente1 = cliente1.obtener_descuento()
descuento_cliente2 = cliente2.obtener_descuento()

# Aplicar promoción
promo = Promocion(10, "pedido")
total_con_descuento = promo.aplicar_descuento(pedido2)

# Consultar historial de pedidos
cliente1.consultar_historial()
cliente2.consultar_historial()

# Cambiar estado de los pedidos
empleado.cambiar_estado_pedido(pedido1, "en preparación")
empleado.cambiar_estado_pedido(pedido2, "entregado")

# Modificar personalización de un producto en un pedido
pedido2.modificar_personalizacion(latte, "extra espresso", "doble")

Inventario inicializado: {'Café Americano': 5, 'Latte': 3, 'Brownie': 4}
Bebida Latte personalizada: leche = almendra
Bebida Latte personalizada: azúcar = sin azúcar
Pedido creado para Lidia Montiel
Café Americano: Disponible
Stock Café Americano: 4
Café Americano agregado al Pedido
Brownie: Disponible
Stock Brownie: 3
Brownie agregado al Pedido
Pedido creado para Daniel Sánchez
Latte: Disponible
Stock Latte: 2
Latte agregado al Pedido
Brownie: Disponible
Stock Brownie: 2
Brownie agregado al Pedido
Descuento aplicado (Cliente Leal): 10.0%
Descuento aplicado (Café Adicto): 15.0%
Promoción creada: 10% en pedido
Total del Pedido: $70.00
Descuento aplicado: Nuevo total $63.00
Lidia Montiel no tiene pedidos.
Daniel Sánchez no tiene pedidos.
Estado del Pedido cambiado a: en preparación
Adrián Morales cambió el estado del Pedido a en preparación
Estado del Pedido cambiado a: entregado
Adrián Morales cambió el estado del Pedido a entregado
Bebida Latte personalizada: extra espresso = doble
Per